# Examen práctica 2
## Aprendizaje Automático II

---

## Importaciones

In [ ]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor

---

## Examen sorpresa resuelto

### 1. Lectura y guardado de datos
Lee los datos de df1 desde el archivo datos_1.csv. Guarda el DataFrame df2 en un csv datos_2.csv

In [ ]:
df1 = pd.read_csv("datos_1.csv")
df2 = pd.to_csv("datos_2.csv")

### 2. Limpieza de valores faltantes
Elimina todas las filas con NaN en ambos DataFrame. Puedes hacerlo de forma segura sin perder el índice o documenta tu decisión.

In [ ]:
df1 = df1.dropna()
df2 = df2.dropna()

### 3. Eliminación de columna innecesaria
En df2 elimina la columna inutil_1

In [ ]:
df2 = df2.drop(columns=["inutil_1"])

### 4. Construcción de df3
Crea df3 que contenga todas las filas de df2 y, para cada id, ñada la información correspondiente de df1.

In [ ]:
df3 = df2.merge(df1, on="id", how="left") # Usamos la columna de df1 de id para hacer un join

### 5. Agregación por moda
En df3, para cada id calcula la moda de cat_3 y añádela como una nueva columna "new_1". Hazlo en una línea y, si hay multimoda, especifica un criterio simple (p.ej., escoger la primera).

In [ ]:
df3["new_1"] = df3.groupby("id")["cat_3"].transform(lambda x: x.mode().loc[0])

### 6. Operaciones de unión y concatenación
Añade una fila nueva a df2 con valores ficticios para cada columna.

In [ ]:
df2.loc[len(df2)] = {"id":999, "cat_3":"nuevo", "cat_4":"x", "num":2}

### 7. Separación de variables
Con df3, separa X (características) de y (target). Debe ser un vector/serie con la columna target; X no debe incluir target.

In [ ]:
y = df3["target"]
X = df3.drop(columns=["target"])

### 8. Pipeline de modelado
Crea un Pipeline que:
- Aplique un preprocesado que transforme las variables categóricas a one-hot encoding usando sklearn.
- Use un árbol de decisión (DecisionTreeClassifier) para predecir target.

Incluye una detección programática de columnas categóricas vs columnas numéricas, división train/test, ajuste del pipeline y una métrica simple.

In [ ]:
# Importaciones
from sklearn.base import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# A partir de la X e y hechas en el ejercicio anterior

# Detección de columnas categóricas vs columas numéricas
cat_cols = X.select_dtypes(include=["object", "category"]).columns
num_cols = X.select_dtypes(include=["number"]).columns

# Preprocesado: [(nombre del bloque, transformador), columnas que recibe, dejar las numéricas tal cual]
preprocesado = ColumnTransformer(transformers=[("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)], remainder="passthrough")

# Árbol de decisión
modelo = DecisionTreeClassifier()

# Pipeline
pipe = Pipeline(steps=[("preprocesado", preprocesado), ("modelo", modelo)])

# División en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_strate=42)

# Ajuste del pipeline
pipe.fit(X_train, y_train)

# Métricas simples
y_pred = pipe.predict(X_test)
acc = accuracy_score(y_test, y_pred)

---

## Códigos examen teórico

### 1. Bagging
Combina varios modelos entrenados en subconjuntos aleatorios del dataset (por bootstrap) para reducir la varianza.
1. Genera B subconjuntos del dataset usando bootstrap (muestras aleatorias con reemplazo).
2. Entrena un modelo base (por ejemplo, un árbol) en cada subconjunto.

Para predecir:
- Regresión: promedio de las predicciones.
- Clasificación: voto mayoritario.

In [ ]:
class Bagging:
    def __init__(self, max_estimators, max_depth):
        self.max_estimators = max_estimators
        self.max_depth = max_depth
        self.estimators = []
        
    def fit(self, x, t):
        ix = np.random.randint(0, x.shape[0], x.shape[0])
        x_samples = x[ix]
        t_samples = t[ix]
        clf = DecisionTreeRegressor(criterion='squared_error', max_depth=self.max_depth)
        clf.fit(x_samples, t_samples)
        self.estimators.append(clf)
        
    def predict(self, x):
        y = np.zeros(x.shape)
        for i in range(self.estimators):
            y += self.estimators[i].predict(x)
        return y / self.estimators

### 2. Random Forest
Es un bagging de árboles donde además se elige aleatoriamente un subconjunto de características en cada división, reduciendo la correlación entre árboles.
1. Igual que Bagging: usa bootstrap para crear subconjuntos.
2. En cada división del árbol, elige un subconjunto aleatorio de variables.
3. Entrena muchos árboles con esas restricciones.
4. Promedia o vota las predicciones.

In [ ]:
class RandomForest:
    def __init__(self, num_estimators, max_depth):
        self.num_estimators = num_estimators
        self.max_depth = max_depth
        self.estimators = []
        
    def fit(self, x, t):
        n_samples, n_features = x.shape
        max_features = int(np.sqrt(n_features))
        
        for _ in range(self.num_estimators):
            ix = np.random.randint(0, n_samples, n_samples)
            x_samples = x[ix]
            t_samples = t[ix]
            
            feature_idx = np.random.choice(n_features, max_features, replace=False)
            
            clf = DecisionTreeRegressor(criterion='squared_error', max_depth=self.max_depth)
            clf.fit(x_samples[:, feature_idx], t_samples)
            clf.feature_inx = feature_idx
            self.estimators.append(clf)
            
    def predict(self, x):
        y = np.zeros(x.shape)
        for i in range(self.num_estimators):
            feature_idx = self.estimators[i].feature_idx
            y += self.estimators[i].predict(x[:, feature_idx])
        return y / self.estimators

### 3. Boosting
Entrena modelos secuencialmente, donde cada uno corrige los errores del anterior, reduciendo sesgo y varianza.
1. Inicializa el modelo con una predicción básica (por ejemplo, la media de y).
2. Calcula los errores del modelo.
3. Entrena un nuevo modelo que corrija esos errores (dando más peso a los ejemplos mal predichos).
4. Combina el nuevo modelo con los anteriores (por suma ponderada).
5. Repite varias iteraciones.

In [ ]:
class Boosting:
   def __init__(self, num_estimators, max_depth):
    self.num_estimators = num_estimators
    self.max_depth = max_depth
    self.estimators = []

   def fit(self, x, t):
    r = t
    for i in range(self.num_estimators):
      # Estimator:
      clf = DecisionTreeRegressor(criterion='squared_error', max_depth=self.max_depth)
      clf.fit(x, r)
      self.estimators.append(clf)

      # Update residual:
      r = r - clf.predict(x)[:, None]

   def predict(self, x):
    y = np.zeros(x.shape[0])
    for i in range(self.num_estimators):
      y += self.estimators[i].predict(x)
    return y

### 4. Gradient Boosting
Versión del boosting que ajusta cada nuevo modelo al gradiente del error (residuos), logrando gran precisión.
1. Empieza con una predicción inicial (p. ej. media de y).
2. Calcula los residuos (gradiente negativo del error).
3. Ajusta un modelo débil (p. ej. árbol pequeño) a esos residuos.
4. Actualiza la predicción con η learning rate.
5. Repite hasta converger o alcanzar N iteraciones.

In [ ]:
class GradientBoosting:
   def __init__(self, num_estimators, max_depth, learning_rate, gradient_function):
    self.num_estimators = num_estimators
    self.max_depth = max_depth
    self.learning_rate = learning_rate
    self.gradient_function = gradient_function
    self.estimators = []

   def fit(self, x, t):
    # Initial estimator:
    clf = DecisionTreeRegressor(criterion='squared_error', max_depth=self.max_depth)
    clf.fit(x, t)
    self.estimators.append(clf)

    # Gradient residual:
    F = clf.predict(x)[:, None]
    r = -self.gradient_function(F, t)

    for i in range(1, self.num_estimators):
      # Estimator:
      clf = DecisionTreeRegressor(criterion='squared_error', max_depth=self.max_depth)
      clf.fit(x, r)
      self.estimators.append(clf)

      # Update residual:
      F = F + self.learning_rate*clf.predict(x)[:, None]
      r = -self.gradient_function(F, t)

   def predict(self, x):
    y = self.estimators[0].predict(x)
    for i in range(1, self.num_estimators):
      y += self.learning_rate*self.estimators[i].predict(x)
    return y

---

## Apuntes teoría

### 1. ¿Qué métricas hay que usar con cada tipo de problema?

- En problemas de regresión: MSE, MAE, RMSE, R2.
- En problemas de clasificación: Accuracy, Precision, Recall, F1-score.

### 2. ¿Cuándo se usa stratify?

- En problemas de clasificación porque la variable objetivo es discreta (son clases).
- Cuando la variable objetivo está muy desbalanceada (por ejemplo 90% clase A y 10% clase B).

### 3. ¿Cuándo se tienen que escalar los datos?

- En problemas de clasificación solo si usa distancias como en k-NN.
- En problemas de regresión en la mayoría de casos, especialmente si las variables no están en la misma escala/unidad y si usamos regresión lineal/ridge/lasso.
- Cuando los atributos numéricos tienen escalas muy diferentes y el modelo depende de distancias o gradientes.
- No es necesario escalar en Árboles de decisión, Random Forest o Gradient Boosting.
- Por ejemplo: StandardScaler o MinMaxScaler.

### 4. ¿Que hace el random_state de train_test_split?

- Fija la semilla del generador aleatorio usado por la función para dividir los datos en entrenamiento y prueba.
- Permite reproducibilidad en los experimentos.

---

## Códigos de la práctica interesantes

### 1. Descargar datos de sklearn

In [ ]:
from sklearn.datasets import fetch_california_housing
data = fetch_california_housing()
X, y = data.data, data.target


### 2. Hacer un selector

In [ ]:
from sklearn.feature_selection import SelectFromModel

selector = SelectFromModel(decision_tree, threshold=0.01, prefit=True) # type: ignore
X_selected = selector.transform(X)

print(f"Número de variables originales: {X.shape[1]}")
print(f"Número de variables seleccionadas: {X_selected.shape[1]}")

### 3. Preprocesado

In [ ]:
# Imprimir el total de valores nulos
from regression_tree import RegressionTree


df = pd.DataFrame(X, columns=data.feature_names)
print(df.isnull().sum())      

# Ver generalmente el dataset
print(f"Shape X: {X.shape}")
print(f"Shape y: {y.shape}")
print(df.describe())

# Entrenamos el árbol óptimo encontrado en el apartado 1
tree = RegressionTree(max_depth=best_depth, min_samples_split=5) # type: ignore
tree.fit(X_train, y_train)

# Seleccionamos dos muestras del conjunto de test
x1 = X_test[0]
x2 = X_test[1]

### 4. Pipelines

In [ ]:
from sklearn.discriminant_analysis import StandardScaler
from sklearn.linear_model import LogisticRegression

# Definir pipeline con escalado y regresión logística
pipe = Pipeline([
    ('scaler', StandardScaler()),          # paso 1: escalado
    ('logreg', LogisticRegression())       # paso 2: modelo
])

# Entrenar pipeline
pipe.fit(X_train, y_train)

# Predecir y evaluar
y_pred = pipe.predict(X_test)
acc_base = accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.ensemble import BaggingClassifier

# Pipeline con Bagging
bagging_model = BaggingClassifier(
    estimator=Pipeline([
        ('scaler', StandardScaler()),
        ('logreg', LogisticRegression())
    ]),
    n_estimators=50,       # número de modelos base
    max_samples=0.8,       # cada modelo ve el 80% del train (bootstrapping)
    bootstrap=True,        # activamos bootstrapping
    n_jobs=-1,             # usar todos los núcleos
    random_state=42
)

# Entrenar pipeline
bagging_model.fit(X_train, y_train)

# Predecir y evaluar
y_pred_bag = bagging_model.predict(X_test)
acc_bag = accuracy_score(y_test, y_pred_bag)